# 25.3 — Risk-Reduction Frontier (Efficiency Curve)

This companion notebook builds the **efficiency frontier** used in Section 25.3.

You will:
1) load an initiative pack (Fleet + PMP)
2) compute efficiency (risk reduction per dollar)
3) sort initiatives by efficiency (descending)
4) compute cumulative investment and cumulative risk reduction
5) plot the **frontier curve** and export figures at **300 dpi**

Outputs are saved into `outputs_25_3/`.


In [ ]:
# --- Imports (Google Colab / local Python) ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO

np.random.seed(42)


## 1) Load initiative pack
The CSV below is embedded for convenience. If you prefer, replace the embedded text with:
```python
df = pd.read_csv('25_3_initiatives.csv')
```


In [ ]:
csv_text = '''Initiative,Scope,Investment_USD,Risk_Reduction_USD
Condition monitoring expansion,Fleet,45000,55000
Critical spares optimization,Fleet,30000,32000
Reliability-centered maintenance refresh,Fleet,25000,26000
Precision alignment program,Fleet,40000,34000
Operator reliability training,Fleet,20000,15000
Lubrication excellence,Fleet,18000,12000
Digital analytics platform,Fleet,50000,28000
Design improvement studies,Fleet,60000,30000
Seal system upgrade,PMP,28000,36000
Cavitation monitoring,PMP,18000,20000
Pump alignment correction,PMP,22000,21000
Spare pump strategy,PMP,35000,30000
Predictive vibration sensors,PMP,26000,19000
Procedure standardization,PMP,14000,9000
Hydraulic re-rating,PMP,30000,15000
'''
df = pd.read_csv(StringIO(csv_text))
df.head()


## 2) Compute efficiency and prepare the frontier table
Definitions:
- **Efficiency** = Risk_Reduction / Investment
- **Cumulative Investment** = running sum of Investment after sorting by efficiency (descending)
- **Cumulative Risk Reduction** = running sum of Risk_Reduction after sorting


In [ ]:
df = df.copy()
df["Efficiency"] = df["Risk_Reduction_USD"] / df["Investment_USD"]

def prepare_scope(d):
    d = d.sort_values("Efficiency", ascending=False).reset_index(drop=True)
    d["Cum_Investment"] = d["Investment_USD"].cumsum()
    d["Cum_Risk_Reduction"] = d["Risk_Reduction_USD"].cumsum()
    return d

fleet = prepare_scope(df[df["Scope"] == "Fleet"])
pmp   = prepare_scope(df[df["Scope"] == "PMP"])

fleet, pmp


## 3) Plot: Efficiency frontier for each scope
X-axis: cumulative investment
Y-axis: cumulative risk reduction


In [ ]:
import os
out_dir = "outputs_25_3"
os.makedirs(out_dir, exist_ok=True)

def plot_frontier(scope_df, title, out_png):
    plt.figure()
    plt.plot(scope_df["Cum_Investment"], scope_df["Cum_Risk_Reduction"], marker="o")
    plt.title(title)
    plt.xlabel("Cumulative investment (USD)")
    plt.ylabel("Cumulative risk reduction (USD)")
    plt.grid(True)
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.show()

plot_frontier(
    fleet,
    "Efficiency Frontier — Fleet (sorted by risk-reduction efficiency)",
    os.path.join(out_dir, "25_3_frontier_Fleet.png")
)

plot_frontier(
    pmp,
    "Efficiency Frontier — Pumps (PMP) (sorted by risk-reduction efficiency)",
    os.path.join(out_dir, "25_3_frontier_PMP.png")
)


## 4) Plot: Combined portfolio frontier (Fleet + PMP)
A single enterprise curve: combine initiatives across scopes and sort globally by efficiency.


In [ ]:
portfolio = df.sort_values("Efficiency", ascending=False).reset_index(drop=True)
portfolio["Cum_Investment"] = portfolio["Investment_USD"].cumsum()
portfolio["Cum_Risk_Reduction"] = portfolio["Risk_Reduction_USD"].cumsum()

plot_frontier(
    portfolio,
    "Efficiency Frontier — Combined Portfolio (Fleet + PMP)",
    os.path.join(out_dir, "25_3_frontier_Portfolio.png")
)

portfolio.head(10)


## 5) Optional: Flag a practical ‘stop’ point (diminishing returns)
A simple rule is to stop funding once efficiency drops below a threshold.


In [ ]:
threshold = 0.75  # edit to your organization’s funding rule
portfolio["Below_Threshold"] = portfolio["Efficiency"] < threshold

portfolio[["Initiative","Scope","Investment_USD","Risk_Reduction_USD","Efficiency","Below_Threshold"]]


## 6) Export tables for the companion repo
Exports:
- sorted frontier tables for Fleet, PMP, and Portfolio


In [ ]:
fleet_csv = os.path.join(out_dir, "25_3_frontier_table_Fleet.csv")
pmp_csv = os.path.join(out_dir, "25_3_frontier_table_PMP.csv")
portfolio_csv = os.path.join(out_dir, "25_3_frontier_table_Portfolio.csv")

fleet.to_csv(fleet_csv, index=False)
pmp.to_csv(pmp_csv, index=False)
portfolio.to_csv(portfolio_csv, index=False)

fleet_csv, pmp_csv, portfolio_csv
